In [1]:
"""
Synthetic Signal Generation Script
-------------------------------------
For the Nature Reviews Methods Primer

This script generates a synthetic time-series signal by combining various components to model complex,
dynamic behaviors often observed in natural systems. The steps involved in the signal generation are:

1. Combining five sinusoidal waves, each with specified frequencies, amplitudes, and phases.
2. Introducing a time-varying amplitude for the combined signal to enhance its dynamism,
   achieved by modulating the amplitudes of individual waves with an envelope function.
3. Applying a nonlinear transformation to the waveform to introduce nonlinearity.
4. Adding transient oscillations with specific frequency, amplitude, and duration, 
   starting at a specified time.
5. Incorporating a weak signal component with a specific frequency and amplitude.
6. Adding a quasi-periodic perturbation, modulated at a secondary frequency.
7. Adding random noise to simulate real-world conditions.

The final synthetic signal is saved as a FITS file for further analysis.
"""
import numpy as np # type: ignore
from astropy.io import fits # type: ignore

# Appendix flag for modifying parameters
appendix1 = False  # Multiply noise amplitude and nonlinear factor by 4

# Parameters for the synthetic wave components
num_waves = 5
freq = np.array([5.0, 12.0, 15.0, 18.0, 25.0])  # Frequencies of the five waves (in Hz)
amp = np.array([1.0, 0.5, 0.8, 0.3, 0.6])       # Amplitudes of the five waves
phase = np.array([0.0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi])  # Initial phases (radians)

# Parameters for the amplitude envelope
envelope_freq = 0.2  # Frequency of the amplitude envelope modulation (in Hz)
envelope_amplitude = 0.5  # Amplitude of the amplitude envelope modulation

# Parameters for transient oscillations
transient_freq = 2.0  # Frequency of the transient oscillations (in Hz)
transient_amplitude = 0.6  # Amplitude of the transient oscillations
transient_duration = 2.0  # Duration of the transient oscillations (in seconds)
transient_start_time = 3.0  # Start time of the transient oscillations (in seconds)

# Parameters for weak signals
weak_signal_freq = 33.0  # Frequency of the weak signal (in Hz)
weak_signal_amplitude = 0.1  # Amplitude of the weak signal

# Parameters for quasi-periodic signatures
quasi_periodic_freq = 10.0  # Base frequency of the quasi-periodic signature (in Hz)
quasi_periodic_amplitude = 0.3  # Amplitude of the quasi-periodic signature
quasi_periodic_modulation_freq = 0.5  # Modulation frequency of the quasi-periodic signature (in Hz)

# Parameters for nonlinear transformation
if appendix1:
    nonlinear_factor = 0.2
else:
    nonlinear_factor = 0.1

# Parameters for noise
if appendix1:
    noise_amplitude = 0.4
else:    
    noise_amplitude = 0.2

# Sampling parameters
sampling_rate = 100.0  # Sampling rate (in Hz)
duration = 10.0  # Duration of the signal (in seconds)

# Generate time vector
time = np.arange(0, duration, 1/sampling_rate)

# Generate synthetic wave components
synthetic_signal = np.zeros_like(time)
for i in range(num_waves):
    amp_modulation = envelope_amplitude * np.sin(2 * np.pi * envelope_freq * time)
    synthetic_signal += (amp[i] + amp_modulation) * np.sin(2 * np.pi * freq[i] * time + phase[i])

# Apply nonlinear transformation
synthetic_signal += nonlinear_factor * synthetic_signal**2

# Add transient oscillation
transient_indices = np.where((time >= transient_start_time) & (time <= transient_start_time + transient_duration))
synthetic_signal[transient_indices] += transient_amplitude * np.sin(2 * np.pi * transient_freq * time[transient_indices])

# Add weak signal
synthetic_signal += weak_signal_amplitude * np.sin(2 * np.pi * weak_signal_freq * time)

# Add quasi-periodic perturbation
quasi_periodic_signature = quasi_periodic_amplitude * np.sin(2 * np.pi * quasi_periodic_freq * time) * \
                           np.sin(2 * np.pi * quasi_periodic_modulation_freq * time)
synthetic_signal += quasi_periodic_signature

# Add noise
noise = noise_amplitude * np.random.normal(0, 1, len(synthetic_signal))
synthetic_signal_with_noise = synthetic_signal + noise

# Save the results to a FITS file
if appendix1:
    file_suffix = "_large_noise"
else:
    file_suffix = ""
fits_file = f"Synthetic_Data/NRMP_signal_1D{file_suffix}.fits"

# Write synthetic signal to FITS file
hdu_signal = fits.PrimaryHDU(synthetic_signal_with_noise)
hdu_signal.header["TIME"] = "SEE EXT=1 OF THE FITS FILE"
hdu_signal.writeto(fits_file, overwrite=True)

# Write time array to FITS file
hdu_time = fits.ImageHDU(time)
hdul = fits.HDUList([hdu_signal, hdu_time])
hdul.writeto(fits_file, overwrite=True)